In [1]:
import numpy as np
import os

print(os.getcwd())

file_path = "data/ASLDVS/ASL_DVS-20250718T000137Z-1-001/ASL_DVS/subject1/l.aedat"

# 1) Read and skip the text header
with open(file_path, 'rb') as f:
    header = f.read(1_000_000).split(b'\r\n')
header_size = sum((len(line) + 2) for line in header if line.startswith(b'#'))

# 2) Read only the first 5 events (10 uint32 values)
raw = np.fromfile(file_path, dtype=np.uint32, offset=header_size, count=20)
print(raw)
events = raw.reshape(-1, 2)   # shape (5, 2): [address, timestamp]

# 3) Decode each address → (x, y, polarity)
addrs = events[:, 0]
ts    = events[:, 1]
xs    = (addrs & 0x7F).astype(int)
ys    = ((addrs >> 7) & 0x7F).astype(int)
ps    = ((addrs >> 15) & 0x1).astype(int)

# 4) Display
print("First 5 events (timestamp, x, y, polarity):")
for t, x, y, p in zip(ts, xs, ys, ps):
    print(f"{t:10d},  {x:2d},  {y:2d},  {p}")

c:\Users\22390013@students.ltu.edu.au\OneDrive - LA TROBE UNIVERSITY\Projects\EventCamera\classification


FileNotFoundError: [Errno 2] No such file or directory: 'data/ASLDVS/ASL_DVS-20250718T000137Z-1-001/ASL_DVS/subject1/l.aedat'

In [ ]:
import tonic
from tonic.datasets import ASLDVS
from torch.utils.data import DataLoader

# 1) Instantiate the dataset (it will download + extract into ./data/ASLDVS if needed)
dataset = ASLDVS(save_to="./data/ASLDVS")

# 2) Number of samples
print("Total samples:", len(dataset))

# 3) Number of classes and class names
print("Number of classes:", len(dataset.classes))
print("Classes:", dataset.classes)

# 4) Peek at the first sample
events, label = dataset[0]
print("\nFirst sample label index:", label)
print("First sample class name:", dataset.classes[label])

# 5) Event array structure
print("\nEvents array shape:", events.shape)
print("Event dtype fields:", events.dtype.names)

# 6) First five events (structured array)
print("\nFirst 5 events:")
print(events[:5])

# 7) (Optional) using a DataLoader
loader = DataLoader(dataset, batch_size=1, shuffle=True)
events_b, label_b = next(iter(loader))
print("\nBatch sample — events tensor shape:", events_b.shape, "label:", label_b)
